In [1]:
import pickle as pkl
import lxml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
import importlib
import time

from tensorflow.keras.datasets import imdb
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Conv1D, Flatten, MaxPooling1D,\
                        AveragePooling1D, Concatenate, LeakyReLU, Embedding,\
                        GlobalMaxPooling1D,GlobalAveragePooling1D,GaussianNoise,BatchNormalization,Add, ZeroPadding1D


importlib.reload(tensorflow)

<module 'tensorflow' from '/miniconda/lib/python3.6/site-packages/tensorflow/__init__.py'>

In [2]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()

In [3]:
tqdm.pandas()

In [4]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    NOTE: Taken from sklearn example
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = [0,1]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    
    #ax.figsize((10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [5]:
drugbankReader = pd.read_csv('../data/intermediate/interactions_preprocessed.csv', chunksize = 1000, iterator = True, converters={"target_gene_encoded_padded": \
                                                                                     lambda x: np.array(x.strip("[]").replace("'","").split(", ")).astype(int)+1,
                                                                                    "drug_fingerprint_encoded_padded": \
                                                                                    lambda x: np.array(x.strip("[]").replace("'","").split(", ")).astype(int)+1})

In [6]:
def drugBankProcessor(reader):
    train_df = reader.__next__()
    #print(df.head())
    train_df_y=list(train_df["Y"])
    train_df_tge_len=list(train_df['target_gene_encoded_len'])
    train_df_dfngprnt_len=list(train_df['drug_fingerprint_encoded_len'])
    train_X0=train_df["target_gene_encoded_padded"].tolist()
    train_X1=train_df["drug_fingerprint_encoded_padded"].tolist()
    train_X0_df=pd.DataFrame(train_X0)
    train_X1_df=pd.DataFrame(train_X1)
    train_drugid=train_df["drug_id"].tolist()
    train_gene_id=train_df["gene_id"].tolist()
    train_X0_array=train_X0_df.values
    print(train_X0_df.shape)
    train_X1_array=train_X1_df.values
    del train_df
    train_df=pd.concat([train_X0_df,train_X1_df], axis=1)
    train_df["Y"]=train_df_y
    del train_df_y
    train_df['target_gene_encoded_padded']=train_df_tge_len
    del train_df_tge_len
    train_df['drug_fingerprint_encoded_padded']=train_df_dfngprnt_len
    del train_df_dfngprnt_len
    train_df=train_df.astype('int16')
    train_df['drug_id']=train_drugid
    del train_drugid
    train_df['gene_id']=train_gene_id
    del train_gene_id
    del train_X0_array
    del train_X1_array
    return(train_df)
    

In [7]:
drugbank1=drugBankProcessor(drugbankReader)

(1000, 21222)


In [9]:
pd.set_option('display.max_rows', 6000)

In [10]:
drugbank1.head()

,0,1,2,3,4,5,6,7,8,9,...,9496,9497,9498,9499,9500,Y,target_gene_encoded_padded,drug_fingerprint_encoded_padded,drug_id,gene_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,303,916,DB02704,BE0004010
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,957,644,DB06684,BE0001790
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,600,195,DB03811,BE0001167
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,834,207,DB02095,BE0004952
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,6546,1216,DB03698,BE0002359


In [ ]:

for chunk in tqdm(drugbankReader):
    chunk_df=drugBankProcessor(drugbankReader)
    print(chunk_df)
    drugbank1=pd.concat([drugbank_bank1,chunk_df], axis=0).reset_index(drop=True)
    del chunk_df

# Training Prep

In [105]:
drugbank_processed=pd.read_pickle('../data/intermediate/interactions_preprocessed_chunk1.pkl')

In [106]:
drugbank_y=drugbank_processed["Y"]

In [109]:
drugbank_processed=drugbank_processed.drop(["Y"], axis=1)

In [110]:
drugbank_processed.columns=list(range(30727))

In [121]:
drop_columns=[]
for i in tqdm(range(len(columns)-8)):
    a=drugbank_processed[i].sum(axis=0)
    if a<=200:
        drop_columns.append(i)
    


In [122]:
drop_columns.append(30725)

In [123]:
drop_columns.append(30726)

In [124]:
drugbank_processed.head()

,0,1,2,3,4,5,6,7,8,9,...,30717,30718,30719,30720,30721,30722,30723,30724,30725,30726
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,303,916,DB02704,BE0004010
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,957,644,DB06684,BE0001790
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,600,195,DB03811,BE0001167
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,834,207,DB02095,BE0004952
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,6546,1216,DB03698,BE0002359


In [187]:
train_X0=drugbank_processed.drop(drop_columns, axis=1)

In [127]:
train_X0_array=train_X0.values

In [188]:
train_X0.shape

(42000, 16537)

In [129]:
from sklearn.decomposition import PCA

In [130]:
pca=PCA(n_components=100)

In [131]:
pca.fit(train_X0_array)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [132]:
pca_transformed_train_X0_array=pca.transform(train_X0_array)

In [134]:
def create_fc_model(input_shape, nb_classes, nb_layers, layer_width, dropout, activation):
    model = Sequential([
  Dense(64, activation='relu', input_shape=(input_shape,)),
  Dense(64, activation='relu'),
  Dense(1, activation='softmax'),
])
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model 

In [135]:
new_model=create_fc_model(100,2,3,50,.2,'relu')

In [136]:
new_model.fit(
  pca_transformed_train_X0_array, # training data
  np.array(drugbank_y), # training targets
  epochs=50,
  batch_size=300,
)

Epoch 1/50
42000/42000 [==============================] - 1s 16us/step - loss: nan - acc: 0.1581
Epoch 2/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 3/50
42000/42000 [==============================] - 1s 16us/step - loss: nan - acc: 0.1581
Epoch 4/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 5/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 6/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 7/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 8/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 9/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 10/50
42000/42000 [==============================] - 0s 9us/step - loss: nan - acc: 0.1581
Epoch 11/50
42000/42000 [============

In [164]:
import deepchem as dc
model = dc.models.SklearnModel(model_instance=LogisticRegression)

In [165]:
dataset=pd.DataFrame()

In [166]:
dataset.X=pca_transformed_train_X0_array

/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [167]:
dataset.y=drugbank_y

/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [168]:
dataset.w=drugbank_y

/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [177]:

import numpy as np

data = np.random.random((4, 4))
labels = np.random.random((4,)) 
from deepchem.data.datasets import NumpyDataset

dataset = NumpyDataset(pca_transformed_train_X0_array, np.array(drugbank_y))

In [179]:
dataset

array([0, 0, 0, ..., 1, 1, 1], dtype=int16)

In [181]:
import tensorflow as tf
from tensorflow.keras.layers import Reshape, Conv2D, Flatten, Dense, Softmax

In [189]:

keras_model = tf.keras.Sequential([
    Reshape((16537, 1)),
    Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu),
    Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu),
    Flatten(),
    Dense(1024, activation=tf.nn.relu),
    Dense(10),
    Softmax()
])
model = dc.models.Model(keras_model)

In [198]:

from tensorflow.examples.tutorials.mnist import input_data

In [199]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use urllib or similar directly.


...

Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.


Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:

train = dc.data.NumpyDataset(mnist.train.images, mnist.train.labels)
valid = dc.data.NumpyDataset(mnist.validation.images, mnist.validation.labels)

In [210]:
keras_model = tf.keras.Sequential([
    Reshape((28, 28, 1)),
    Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu),
    Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu),
    Flatten(),
    Dense(1024, activation=tf.nn.relu),
    Dense(10),
    Softmax()
])
keras_model.compile('sgd', metrics=['sparse_categorical_accuracy'])

In [211]:
model = dc.models.Model(keras_model)

In [215]:
model.fit_on_batch(dataset.X,dataset.y,dataset.w)

NotImplementedError: Each model is responsible for its own fit_on_batch method.

In [218]:
dataset.w

array([1, 1, 1, ..., 1, 1, 1], dtype=int16)